In [53]:
from utils import *
from pprint import pprint
from decisiontree import *

In [54]:
house = importfile('hw3_house_votes_84.csv', ',')
housecategory = {}
for i in house[0]:
    housecategory[i] = 'categorical'
housecategory["class"] = 'class'
housedata = np.array(house[1:]).astype(float)

# print(len(housedata))
# print(housecategory)

#class in last column

In [55]:
wine = importfile('hw3_wine.csv', '\t')
winecategory = {}
for i in wine[0]:
    winecategory[i] = 'numerical'
winecategory["# class"] = 'class'
winedata = np.array(wine[1:]).astype(float)
# winedata = np.array([[float(item) for item in onelist] for onelist in wine[1:]])

# print(len(winedata))
# print(winecategory)

#class in first column

In [56]:
cancer = importfile('hw3_cancer.csv', '\t')
cancercategory = {}
for i in cancer[0]:
    cancercategory[i] = 'numerical'
cancercategory["Class"] = 'class'
cancerdata = np.array(cancer[1:]).astype(float)

#print(len(cancerdata))
#print(cancercategory)

#class in last column

In [57]:
cmc = importfile('cmc.data', ',')
cmccategory = {"Wife's age":"numerical","Wife's education":"categorical",
"Husband's education":"categorical","Number of children ever born":"numerical",
"Wife's religion":"binary","Wife's now working?":"binary",
"Husband's occupation":"categorical","Standard-of-living index":"categorical",
"Media exposure":"binary","Contraceptive method used":"class"}
cmcdata = np.array(cmc).astype(int)

# print(len(cmcdata))
# print(cmccategory)

#class in last column

In [58]:
def cartbestseperate(dataset, attributes:dict):
    # dataset in is the dataset by row. 
    # attributes is the dictionary of attributes:type 
    # types: numerical, categorical, binary.
    datasetbycolumn = dataset.T
    classindex = list(attributes.values()).index("class")
    originalgini = gini(datasetbycolumn[classindex])
    smallestgini = originalgini
    thresholdvalue = -1

    i = 0
    bestattribute = {list(attributes.keys())[i]:attributes[list(attributes.keys())[i]]}
    attributesinuse = list(attributes.keys())[1:] if (classindex == 0) else list(attributes.keys())[:classindex]
    # datasetinuse = datasetbycolumn[1:] if (classindex == 0) else datasetbycolumn[:classindex]

    for attribute in attributesinuse:
        idx = i+1 if classindex == 0 else i

        if attributes[attribute] == "categorical" or attributes[attribute] == "binary":
            listofkeys = list(Counter(datasetbycolumn[idx]).keys())
            listofcategory = [] # this is the list of categorical values.
            
            for key in listofkeys:
                indexlist = [idex for idex, element in enumerate(datasetbycolumn[idx]) if element == key]
                category = np.array(datasetbycolumn[classindex][indexlist])
                listofcategory.append(category)

            currentgini = 0

            for ctgry in listofcategory:
                a = len(ctgry)/len(datasetbycolumn[idx]) # This is probability
                currentgini += a * gini(ctgry)

            if currentgini < smallestgini:
                smallestgini = currentgini
                bestattribute = {attribute:attributes[attribute]}
            
        elif attributes[attribute] == "numerical":
            datasetsort = datasetbycolumn.T[datasetbycolumn.T[:,idx].argsort(kind='quicksort')].T
            currentthreshold = (datasetsort[idx][1]+datasetsort[idx][0])/2
            k = 1
            while k < len(datasetsort.T):
                currentthreshold = (datasetsort[idx][k]+datasetsort[idx][k-1])/2
                listofcategory = [datasetsort[classindex][:k],datasetsort[classindex][k:]]
                currentgini = 0

                for ctgry in listofcategory:
                    a = len(ctgry)/len(datasetbycolumn[idx]) # This is probability
                    currentgini += a * gini(ctgry)

                if currentgini < smallestgini:
                    smallestgini = currentgini
                    thresholdvalue = currentthreshold
                    bestattribute = {attribute:attributes[attribute]}    
                k += 1
        i += 1

    # set first attribution dictionary {key:type} to the best attributes.
    return bestattribute, thresholdvalue, smallestgini

In [59]:
def id3bestseperate(dataset, attributes:dict):
    # dataset in is the dataset by row. 
    # attributes is the dictionary of attributes:type 
    # types: numerical, categorical, binary.
    datasetbycolumn = dataset.T
    classindex = list(attributes.values()).index("class")
    originalentrophy = entropy(datasetbycolumn[classindex])
    smallestentrophy = originalentrophy
    thresholdvalue = -1

    i = 0
    bestattribute = {list(attributes.keys())[i]:attributes[list(attributes.keys())[i]]}
    attributesinuse = list(attributes.keys())[1:] if (classindex == 0) else list(attributes.keys())[:classindex]
    # datasetinuse = datasetbycolumn[1:] if (classindex == 0) else datasetbycolumn[:classindex]

    for attribute in attributesinuse:
        idx = i+1 if classindex == 0 else i

        if attributes[attribute] == "categorical" or attributes[attribute] == "binary":
            listofkeys = list(Counter(datasetbycolumn[idx]).keys())
            listofcategory = [] # this is the list of categorical values.
            
            for key in listofkeys:
                indexlist = [idex for idex, element in enumerate(datasetbycolumn[idx]) if element == key]
                category = np.array(datasetbycolumn[classindex][indexlist])
                listofcategory.append(category)

            entropynow = 0

            for ctgry in listofcategory:
                a = len(ctgry)/len(datasetbycolumn[idx]) # This is probability
                entropynow += a * entropy(ctgry)

            if entropynow < smallestentrophy:
                smallestentrophy = entropynow
                bestattribute = {attribute:attributes[attribute]}
            
        elif attributes[attribute] == "numerical":
            datasetsort = datasetbycolumn.T[datasetbycolumn.T[:,idx].argsort(kind='quicksort')].T
            currentthreshold = (datasetsort[idx][1]+datasetsort[idx][0])/2
            k = 1
            while k < len(datasetsort.T):
                currentthreshold = (datasetsort[idx][k]+datasetsort[idx][k-1])/2
                listofcategory = [datasetsort[classindex][:k],datasetsort[classindex][k:]]
                entropynow = 0

                for ctgry in listofcategory:
                    a = len(ctgry)/len(datasetbycolumn[idx]) # This is probability
                    entropynow += a * entropy(ctgry)

                if entropynow < smallestentrophy:
                    smallestentrophy = entropynow
                    thresholdvalue = currentthreshold
                    bestattribute = {attribute:attributes[attribute]}    
                k += 1
        i += 1

    gain = originalentrophy-smallestentrophy
    # set first attribution dictionary {key:type} to the best attributes.
    return bestattribute, thresholdvalue, gain

In [60]:
def dropbyindex(data, category, listindex):
    newdata = np.delete(data.T, listindex,0).T
    keytoremove = [list(category.keys())[i] for i in listindex]
    newcategory = category.copy()
    [newcategory.pop(key) for key in keytoremove]
    return newdata, newcategory

In [61]:
removeindex0= [1,2,3,5,6,7,8]
data1,category1=dropbyindex(winedata,winecategory,removeindex0)
print(len(data1))
bestattribute, thresholdvalue, gain = id3bestseperate(data1,category1)
print(bestattribute, thresholdvalue, gain)

178
{'12': 'numerical'} 2.475 0.6173242577120465


In [62]:
removeindex= [7,13,12,10]
kdata1,kcategory1=dropbyindex(winedata,winecategory,removeindex)
bagini, tgini, vgini = cartbestseperate(kdata1,kcategory1)
baent, tent, vent = id3bestseperate(kdata1,kcategory1)
print("entropy best attribute is: ", baent, ", and entropy threshold is: ", tent, ", gain is: ", vent)
print("gini best attribute is: ", bagini, ", and gini threshold is: ", tgini, ", gini is: ", vgini)


entropy best attribute is:  {'1': 'numerical'} , and entropy threshold is:  12.78 , gain is:  0.5483727619642427
gini best attribute is:  {'1': 'numerical'} , and gini threshold is:  12.78 , gini is:  0.43102824014013796


In [63]:
# Decision Tree
def decisiontree(dataset: np.array, dictattributes: dict, algortype: str ='id3'):
    datasetcopy = np.copy(dataset).T # dataset copy is by colomn. 
    dictattricopy = dictattributes.copy()
    classindex = list(dictattricopy.values()).index("class")

    def processbest(algor):
        if algor == "cart" or algor == "gini":
            return cartbestseperate(datasetcopy.T, dictattricopy)
        else: # algor == "id3" or algor == "infogain"
            return id3bestseperate(datasetcopy.T, dictattricopy)

    node = Treenode(label=-1,type="decision")

    node.majority = majority(datasetcopy[classindex])

    if same(datasetcopy[classindex]):
        node.type = "leaf"
        node.label = datasetcopy[classindex][0]
        return node
    
    if len(dictattricopy) == 0:
        node.type = "leaf"
        node.label = majority(datasetcopy[classindex])
        return node

    bestattributedict,thresholdval = processbest(algortype)[:2]
    bestattributename = list(bestattributedict.keys())[0]
    bestattributetype = bestattributedict[bestattributename]
    node.testattributedict = bestattributedict
    node.datatype = bestattributetype
    node.testattribute = bestattributename
    node.threshold = thresholdval
    bindex = list(dictattricopy.keys()).index(list(bestattributedict.keys())[0])

    subdatalists = []
    if bestattributetype == "numerical":
        sortedcopy = datasetcopy.T[datasetcopy.T[:,bindex].argsort(kind='quicksort')].T
        splitindex = 0
        for numericalvalue in sortedcopy[bindex]:
            if numericalvalue > thresholdval:
                break
            else:
                splitindex += 1
        subdatalistraw = [sortedcopy.T[:splitindex].T,sortedcopy.T[splitindex:].T]
        for subdata in subdatalistraw:
            subdatav = np.delete(subdata,bindex,0)
            subdatalists.append(subdatav.T)
    else:
        bigv = list(Counter(datasetcopy[bindex]).keys()) # this is the all the categories of the test attribute left.
    
        for smallv in bigv:
            index = [idx for idx, element in enumerate(datasetcopy[bindex]) if element == smallv]
            subdatav = np.array(datasetcopy.T[index]).T
            subdatav = np.delete(subdatav,bindex,0)  # I delete the column I already used using bindex as reference. 
            # Then, later, pop the same index from list attribute.
            subdatalists.append(subdatav.T) # list of nparrays of target/label/categories.

    dictattricopy.pop(bestattributename)
    
    edge = {}
    sdindex = 0
    for subvdata in subdatalists:
        if subvdata.size == 0:
            node.type = "leaf"
            node.label = -1
            node.threshold = thresholdval
            return node

        subtree = decisiontree(subvdata, dictattricopy, algortype)
        if bestattributetype == 'numerical':
            attributevalue = "<=" if sdindex == 0 else ">"
        else:
            attributevalue = bigv[sdindex]

        edge[attributevalue] = subtree
        sdindex += 1

    node.edge = edge

    return node

In [64]:
cartbestseperate(cmcdata,cmccategory)

({'Number of children ever born': 'numerical'}, 1.0, 0.6124220726660862)

In [65]:
tree = decisiontree(cmcdata,cmccategory,'cart')
# classindex = list(housecategory.values()).index("class")
# classindex
# classcount = housedata.T[classindex]
# majority(classcount)

print(tree.edge,tree.threshold,tree.testattribute)

{'<=': <decisiontree.Treenode object at 0x16b4ca350>, '>': <decisiontree.Treenode object at 0x16b4ca2f0>} 1.0 Number of children ever born


In [66]:
teslist = np.array([1,2,3,4,5,6,7,8,9,10])
thold = 4.5
splitindex = 0

for numericalvalue in teslist:
    if numericalvalue > thold:
        break
    splitindex += 1

print(splitindex)

print([teslist[:splitindex],teslist[splitindex:]])

4
[array([1, 2, 3, 4]), array([ 5,  6,  7,  8,  9, 10])]
